In [1]:
import pandas as pd
import requests
import geopy

In [2]:
df = pd.read_csv("C:\\Users\\viven\\Downloads\\Food Delivery dataset\\deliverytime.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  int64  
 3   Delivery_person_Ratings      45593 non-null  float64
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Type_of_order                45593 non-null  object 
 9   Type_of_vehicle              45593 non-null  object 
 10  Time_taken(min)              45593 non-null  int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 3.8+ MB


In [3]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30


In [4]:
indore_df = df[df['Delivery_person_ID'].str.contains("INDORE", na=False, case=False)].copy()
print(indore_df.shape)

indore_df = indore_df[(indore_df != 0).all(axis=1)]
print(indore_df.shape)

(3159, 11)
(3159, 11)


In [6]:
from geopy.distance import geodesic

# Indore city's location
city_lat = 22.719568
city_lon = 75.857727

# Define the distance threshold (in meters)
distance_threshold = 10  # Adjust this as needed, it's in kilometers

# Function to calculate the distance from the city center
def calculate_distance(lat, lon, city_lat, city_lon):
    return geodesic((lat, lon), (city_lat, city_lon)).kilometers

# Calculate the distance for restaurant and delivery locations
indore_df['Restaurant_distance'] = indore_df.apply(lambda row: calculate_distance(row['Restaurant_latitude'], row['Restaurant_longitude'], city_lat, city_lon), axis=1)
indore_df['Delivery_distance'] = indore_df.apply(lambda row: calculate_distance(row['Delivery_location_latitude'], row['Delivery_location_longitude'], city_lat, city_lon), axis=1)

# Drop rows that are beyond the distance threshold for both restaurant and delivery locations
indore_df_filtered = indore_df[(indore_df['Restaurant_distance'] <= distance_threshold) & (indore_df['Delivery_distance'] <= distance_threshold)]

# Drop the distance columns
indore_df_filtered = indore_df_filtered.drop(columns=['Restaurant_distance', 'Delivery_distance'])


In [7]:
indore_df_filtered

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
93,6D19,INDORES12DEL02,38,4.9,22.748060,75.893400,22.768060,75.913400,Drinks,scooter,15
104,6757,INDORES05DEL02,24,4.3,22.727021,75.884167,22.747021,75.904167,Meal,motorcycle,19
133,977D,INDORES19DEL02,27,4.7,22.745536,75.893106,22.765536,75.913106,Buffet,motorcycle,14
273,144D,INDORES18DEL02,22,4.8,22.753839,75.897429,22.773839,75.917429,Drinks,motorcycle,19
...,...,...,...,...,...,...,...,...,...,...,...
45383,7C6A,INDORES14DEL02,39,4.2,22.761593,75.886362,22.781593,75.906362,Buffet,motorcycle,32
45393,7266,INDORES08DEL02,26,4.9,22.725748,75.898497,22.745748,75.918497,Buffet,motorcycle,21
45483,120B,INDORES02DEL02,25,4.0,22.651847,75.881991,22.671847,75.901991,Snack,motorcycle,19
45518,863,INDORES05DEL01,29,4.6,22.727021,75.884167,22.737021,75.894167,Snack,electric_scooter,25


In [8]:
import osmnx as ox
import networkx as nx

df1 = indore_df_filtered.copy()

# Define a function to calculate the shortest path distance
def calculate_shortest_path_distance(row, G):
    restaurant_lat_lon = (row['Restaurant_latitude'], row['Restaurant_longitude'])
    delivery_lat_lon = (row['Delivery_location_latitude'], row['Delivery_location_longitude'])
    
    # Get the nearest nodes to the restaurant and delivery locations
    orig_node = ox.distance.nearest_nodes(G, row['Restaurant_longitude'], row['Restaurant_latitude'])
    dest_node = ox.distance.nearest_nodes(G, row['Delivery_location_longitude'], row['Delivery_location_latitude'])
    
    # Calculate the shortest path distance between the nodes
    shortest_path_distance = nx.shortest_path_length(G, orig_node, dest_node, weight='length')
    
    return shortest_path_distance

# Create a graph from OpenStreetMap data
G = ox.graph_from_point((22.719568, 75.857727), dist=10000, network_type='drive')

# Apply the function to each row in the dataset
df1['driving_distance'] = df1.apply(lambda row: calculate_shortest_path_distance(row, G), axis=1)

In [9]:
df1

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min),driving_distance
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24,3967.774166
93,6D19,INDORES12DEL02,38,4.9,22.748060,75.893400,22.768060,75.913400,Drinks,scooter,15,4745.077512
104,6757,INDORES05DEL02,24,4.3,22.727021,75.884167,22.747021,75.904167,Meal,motorcycle,19,3674.590510
133,977D,INDORES19DEL02,27,4.7,22.745536,75.893106,22.765536,75.913106,Buffet,motorcycle,14,4144.178413
273,144D,INDORES18DEL02,22,4.8,22.753839,75.897429,22.773839,75.917429,Drinks,motorcycle,19,4106.179566
...,...,...,...,...,...,...,...,...,...,...,...,...
45383,7C6A,INDORES14DEL02,39,4.2,22.761593,75.886362,22.781593,75.906362,Buffet,motorcycle,32,4856.619253
45393,7266,INDORES08DEL02,26,4.9,22.725748,75.898497,22.745748,75.918497,Buffet,motorcycle,21,3583.268984
45483,120B,INDORES02DEL02,25,4.0,22.651847,75.881991,22.671847,75.901991,Snack,motorcycle,19,5196.644558
45518,863,INDORES05DEL01,29,4.6,22.727021,75.884167,22.737021,75.894167,Snack,electric_scooter,25,2019.953271


In [10]:
new_df = df1[['Delivery_person_Age', 'Delivery_person_Ratings', 'Type_of_vehicle', 'Time_taken(min)', 'driving_distance']]
new_df

,Delivery_person_Age,Delivery_person_Ratings,Type_of_vehicle,Time_taken(min),driving_distance
0,37,4.9,motorcycle,24,3967.774166
93,38,4.9,scooter,15,4745.077512
104,24,4.3,motorcycle,19,3674.590510
133,27,4.7,motorcycle,14,4144.178413
273,22,4.8,motorcycle,19,4106.179566
...,...,...,...,...,...
45383,39,4.2,motorcycle,32,4856.619253
45393,26,4.9,motorcycle,21,3583.268984
45483,25,4.0,motorcycle,19,5196.644558
45518,29,4.6,electric_scooter,25,2019.953271


In [11]:
new_df['Type_of_vehicle'].unique()

array(['motorcycle ', 'scooter ', 'electric_scooter ', 'bicycle '],
      dtype=object)

In [12]:
new_df['driving_distance'] = new_df['driving_distance'] / 1000

new_df = pd.get_dummies(new_df, columns=['Type_of_vehicle'])

C:\Users\viven\AppData\Local\Temp\ipykernel_17492\3436245263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['driving_distance'] = new_df['driving_distance'] / 1000


In [16]:
new_df.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Time_taken(min)',
       'driving_distance', 'Type_of_vehicle_bicycle ',
       'Type_of_vehicle_electric_scooter ', 'Type_of_vehicle_motorcycle ',
       'Type_of_vehicle_scooter '],
      dtype='object')

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

new_df.columns = new_df.columns.str.strip()

X = new_df[['Delivery_person_Age', 'driving_distance', 'Type_of_vehicle_bicycle', 'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle', 'Type_of_vehicle_scooter']]
y = new_df['Time_taken(min)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)

Root Mean Squared Error: 6.988907842865499
                                  Coefficient
Delivery_person_Age                  0.400301
driving_distance                     0.573013
Type_of_vehicle_bicycle              2.271378
Type_of_vehicle_electric_scooter    -2.456013
Type_of_vehicle_motorcycle           2.203010
Type_of_vehicle_scooter             -2.018375


Coefficient Interpretations:

Delivery_person_Age (0.400301): For each additional year in the delivery person's age, the predicted delivery time increases by approximately 0.40 minutes.

Driving_distance (0.573013): For each additional kilometer of driving distance, the predicted delivery time increases by approximately 0.57 minutes.

Type_of_vehicle_bicycle (2.271378): If the delivery is made using a bicycle (compared to other vehicles), the predicted delivery time increases by approximately 2.27 minutes.

Type_of_vehicle_electric_scooter (-2.456013): If the delivery is made using an electric scooter, the predicted delivery time decreases by approximately 2.46 minutes.

Type_of_vehicle_motorcycle (2.203010): If the delivery is made using a motorcycle, the predicted delivery time increases by approximately 2.20 minutes.

Type_of_vehicle_scooter (-2.018375): If the delivery is made using a scooter, the predicted delivery time decreases by approximately 2.02 minutes.

In summary:

Older delivery personnel and longer driving distances increase delivery time.

Bicycles and motorcycles increase delivery time compared to other vehicles.

Electric scooters and scooters decrease delivery time compared to other vehicles.

RMSE (Root Mean Squared Error) of 6.98 means that, on average, the predictions made by our model are off by approximately 6.98 minutes from the actual values

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

new_df.columns = new_df.columns.str.strip()

X2 = new_df[['Delivery_person_Age', 'Time_taken(min)', 'driving_distance', 'Type_of_vehicle_bicycle', 'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle', 'Type_of_vehicle_scooter']]
y2 = new_df['Delivery_person_Ratings']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model2 = LinearRegression()
model2.fit(X2_train, y2_train)

y2_pred = model2.predict(X2_test)

rmse = np.sqrt(mean_squared_error(y2_test, y2_pred))
print(f'Root Mean Squared Error: {rmse}')

coefficients = pd.DataFrame(model2.coef_, X2.columns, columns=['Coefficient'])
print(coefficients)

Root Mean Squared Error: 0.23052682249256043
                                  Coefficient
Delivery_person_Age                  0.006040
Time_taken(min)                     -0.011051
driving_distance                     0.006932
Type_of_vehicle_bicycle             -0.046450
Type_of_vehicle_electric_scooter     0.042186
Type_of_vehicle_motorcycle          -0.031399
Type_of_vehicle_scooter              0.035664


Coefficient Interpretations:

Time_taken(min) (-0.011051): For each additional minute taken, the predicted rating decreases by approximately 0.011 units. This implies that longer delivery times might lead to lower ratings.

Driving_distance (0.006932): For each additional kilometer in driving distance, the predicted rating increases by approximately 0.007 units. This suggests that longer driving distances might be slightly associated with higher ratings.

Type_of_vehicle_bicycle (-0.046450): If the delivery is made using a bicycle, the predicted rating decreases by approximately 0.046 units compared to other vehicle types.

Type_of_vehicle_electric_scooter (0.042186): If the delivery is made using an electric scooter, the predicted rating increases by approximately 0.042 units compared to other vehicle types.

Type_of_vehicle_motorcycle (-0.031399): If the delivery is made using a motorcycle, the predicted rating decreases by approximately 0.031 units compared to other vehicle types.

Type_of_vehicle_scooter (0.035664): If the delivery is made using a scooter, the predicted rating increases by approximately 0.036 units compared to other vehicle types.

In conclusion:

Older delivery personnel receive slightly higher ratings.

Longer delivery times negatively impact ratings.

Longer driving distances slightly improve ratings.

Bicycles and motorcycles slightly decrease ratings compared to other vehicles.

Electric scooters and scooters slightly improve ratings compared to other vehicles.

The average error in predictions is approximately 0.23 units. Given that rating scales are (1,5) an RMSE of 0.23 suggests that model is quite accurate.